Initiales setup für GPU Verwendung

In [4]:
!wget https://developer.download.nvidia.com/compute/cuda/11.0.3/local_installers/cuda-repo-ubuntu2004-11-0-local_11.0.3-450.51.06-1_amd64.deb
!dpkg -i cuda-repo-ubuntu2004-11-0-local_11.0.3-450.51.06-1_amd64.deb
!apt-key add /var/cuda-repo-ubuntu2004-11-0-local/7fa2af80.pub
!apt-get update
!apt-get install cuda
#!pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

Der Befehl "wget" ist entweder falsch geschrieben oder
konnte nicht gefunden werden.
Der Befehl "dpkg" ist entweder falsch geschrieben oder
konnte nicht gefunden werden.
Der Befehl "apt-key" ist entweder falsch geschrieben oder
konnte nicht gefunden werden.
Der Befehl "apt-get" ist entweder falsch geschrieben oder
konnte nicht gefunden werden.
Der Befehl "apt-get" ist entweder falsch geschrieben oder
konnte nicht gefunden werden.


Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html
  Using cached https://download.pytorch.org/whl/cu113/torch-1.11.0%2Bcu113-cp39-cp39-win_amd64.whl (2186.0 MB)
  Using cached https://download.pytorch.org/whl/cu113/torchvision-0.12.0%2Bcu113-cp39-cp39-win_amd64.whl (5.4 MB)
  Using cached https://download.pytorch.org/whl/cu113/torchaudio-0.11.0%2Bcu113-cp39-cp39-win_amd64.whl (573 kB)
  Attempting uninstall: torch
    Found existing installation: torch 1.12.0
    Uninstalling torch-1.12.0:
      Successfully uninstalled torch-1.12.0


ERROR: Could not install packages due to an OSError: [WinError 5] Zugriff verweigert: 'C:\\Users\\franz.buehler\\Anaconda3\\Lib\\site-packages\\~orch\\lib\\asmjit.dll'
Consider using the `--user` option or check the permissions.



Installieren der Dependencies

In [1]:
!pip install spacy==3.3.1 
!pip install spacy-transformers
#export CUDA_PATH="/opt/nvidia/cuda"
!pip install -U spacy[transformers,cuda113]
#for transformers v4.0.0+ and models that require SentencePiece (e.g., ALBERT, CamemBERT, XLNet, Marian, and T5)
!pip install transformers[sentencepiece]
!pip install spacy-sentence-bert

  Using cached spacy-3.3.1-cp39-cp39-win_amd64.whl (11.7 MB)
  Using cached pathy-0.6.2-py3-none-any.whl (42 kB)
  Using cached catalogue-2.0.7-py3-none-any.whl (17 kB)
  Using cached cymem-2.0.6-cp39-cp39-win_amd64.whl (36 kB)
  Using cached blis-0.7.8-cp39-cp39-win_amd64.whl (6.6 MB)
  Using cached spacy_legacy-3.0.9-py2.py3-none-any.whl (20 kB)
  Using cached typer-0.4.1-py3-none-any.whl (27 kB)
  Using cached srsly-2.4.3-cp39-cp39-win_amd64.whl (448 kB)
  Using cached thinc-8.0.17-cp39-cp39-win_amd64.whl (1.0 MB)
  Using cached preshed-3.0.6-cp39-cp39-win_amd64.whl (112 kB)
  Using cached murmurhash-1.0.7-cp39-cp39-win_amd64.whl (18 kB)
  Using cached spacy_loggers-1.0.2-py3-none-any.whl (7.2 kB)
  Using cached pydantic-1.8.2-cp39-cp39-win_amd64.whl (1.9 MB)
  Using cached wasabi-0.9.1-py3-none-any.whl (26 kB)
  Using cached langcodes-3.3.0-py3-none-any.whl (181 kB)
  Using cached smart_open-5.2.1-py3-none-any.whl (58 kB)
  Attempting uninstall: smart-open
    Found existing instal

In [ ]:
!pip install shap

Laden der Spacy Modelle

In [2]:
!python -m spacy download de_core_news_lg
!python -m spacy download de_dep_news_trf

C:\Users\franz.buehler\Anaconda3\lib\site-packages\cupy\_environment.py:213: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(
C:\Users\franz.buehler\Anaconda3\lib\site-packages\cupy\_environment.py:213: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(
C:\Users\franz.buehler\Anaconda3\lib\site-packages\cupy\_environment.py:213: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(



[+] Download and installation successful
You can now load the package via spacy.load('de_core_news_lg')
[+] Download and installation successful
You can now load the package via spacy.load('de_dep_news_trf')


C:\Users\franz.buehler\Anaconda3\lib\site-packages\cupy\_environment.py:213: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(
C:\Users\franz.buehler\Anaconda3\lib\site-packages\cupy\_environment.py:213: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(
C:\Users\franz.buehler\Anaconda3\lib\site-packages\cupy\_environment.py:213: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(


Bibliotheken im

In [19]:
import spacy
import spacy_sentence_bert
import pandas as pd
import random
from spacy.util import minibatch, compounding
#import shap
from thinc.api import set_gpu_allocator, require_gpu

# Use the GPU, with memory allocations directed via PyTorch.
# This prevents out-of-memory errors that would otherwise occur from competing
# memory pools.
#set_gpu_allocator("pytorch")
#distiluse-base-multilingual-cased-v1

# load one of the models listed at https://github.com/MartinoMensio/spacy-sentence-bert/
nlp = spacy_sentence_bert.load_model('xx_stsb_xlm_r_multilingual')
#nlp.add_pipe('sentence_bert', config={'model_name': 'xx_stsb_xlm_r_multilingual'}) #'T-Systems-onsite/cross-en-de-roberta-sentence-transformer'})

doc1 = nlp("Es muss klar sein, wer wann was wie machen soll. Die Ziele des Computershops müssen klar werden. Am Anfang muss beschrieben werden, wie das Spiel abläuft und worum es genau geht. Es muss klar benannt werden, wofür es im Spiel Punkte gibt und wofür nicht. Auch muss spezifiziert werden, wer Weisungsbefugnisse hat und wer nicht")
doc2 = nlp("Es sollte erklärt werden, was getan werden muss, was die Ziele des Shops sind und wie der Ablauf des Spiels ist.")
#doc2 = nlp("Man muss genau wissen, worum es beim Spiel oder den Aufgaben geht und was wie gewertet wird. Dazu wird am Anfang eine Beschreibung oder Einleitungen benötigt, die die Ziele der Aufgaben deutlich machen.")

#print(doc1.ents, len(doc1.ents))
#print(doc1.tensor)
#shap_values = explainer(doc1.tensor)

x =0;
for sent1 in doc1.sents:
  x+= 1
  y=0;
  for sent2 in doc2.sents:
    y += 1
    print(sent1.similarity(sent2), x, y)
print(doc12.similarity(doc2))


0.5528104305267334 1 1
0.6887466907501221 2 1
0.7276392579078674 3 1
0.5777024626731873 4 1
0.4021310806274414 5 1
0.7874074910561945
